In [1]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

a = 'This was a good movie.'
sid.polarity_scores(a)

a = 'This was the best, most awesome movie EVER MADE!!!'
sid.polarity_scores(a)

a = 'This was the worst film to ever disgrace the screen.'
sid.polarity_scores(a)

a = 'This was the WORST film to ever disgrace the screen.'
sid.polarity_scores(a)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.501, 'neu': 0.499, 'pos': 0.0, 'compound': -0.8415}

In [6]:
"""
Use VADER to analyze Amazon Reviews
For this exercise we're going to apply SentimentIntensityAnalyzer to a dataset of 10,000 Amazon reviews.
Like our movie reviews datasets, these are labeled as either "pos" or "neg". At the end we'll determine
the accuracy of our sentiment analysis with VADER.
"""
import numpy as np
import pandas as pd
import os 
path = "D:/Natural Processing Language/BUAN 6V99 _MARCH_10/BUAN 6V99/Python Data Files"
os.chdir(path)
df = pd.read_csv('amazonreviews.tsv', sep='\t')
df.head()
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [8]:
"""
Clean the data (optional):
Recall that our moviereviews.tsv file contained empty records. Let's check to see if any exist in
amazonreviews.tsv.
"""
# REMOVE NaN VALUES AND EMPTY STRINGS:
df.dropna(inplace=True)

blanks = []  # start with an empty list

for i,lb,rv in df.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df.iloc[0]['review']
df.drop(blanks, inplace=True)
df['label'].value_counts()


neg    5097
pos    4903
Name: label, dtype: int64

In [9]:
#Let's run the first review through VADER
sid.polarity_scores(df.loc[0]['review'])
df.loc[0]['label']

'pos'

In [14]:
"""
Adding Scores and Labels to the DataFrame
In this next section we'll add columns to the original DataFrame to store polarity_score dictionaries, extracted compound scores, and new "pos/neg" labels derived from the compound score. We'll use this last column to perform an accuracy test.
"""

df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))
df['scores'].head()#### sid.polarity_scores(review) has 4 score result in it namely: neg, neu, pos, compound

df['compound']  = df['scores'].apply(lambda x: x['compound'])
df.head()

df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
df.head()

,label,review,scores,compound,comp_score
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


In [17]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(df['label'],df['comp_score'])



0.7091

In [16]:
print(classification_report(df['label'],df['comp_score']))

             precision    recall  f1-score   support

        neg       0.86      0.51      0.64      5097
        pos       0.64      0.91      0.75      4903

avg / total       0.75      0.71      0.70     10000

